In [1]:
%%time

N = 50_000_000

output = [None] * N

for i in range(N):
    if (i + 1) % 15 == 0:
        output[i] = 1337
    elif (i + 1) % 3 == 0:
        output[i] = 17
    elif (i + 1) % 5 == 0:
        output[i] = 42
    else:
        output[i] = i + 1

print(f"{output[0]=}")
print(f"{output[1]=}")
print(f"{output[2]=}")
print(f"{output[N-1]=}")

output[0]=1
output[1]=2
output[2]=17
output[N-1]=42
CPU times: user 7.71 s, sys: 507 ms, total: 8.22 s
Wall time: 8.22 s


In [2]:
%%time

import numpy as np

N = 50_000_000

source_array = np.arange(1, N + 1, dtype=np.int32)
output = np.copy(source_array)

divisible_by_15 = source_array % 15 == 0
output[divisible_by_15] = 1337

divisible_by_3_only = (source_array % 3 == 0) & (~divisible_by_15)
output[divisible_by_3_only] = 17

divisible_by_5_only = (source_array % 5 == 0) & (~divisible_by_15)
output[divisible_by_5_only] = 42

print(output)

[       1        2       17 ...       17 49999999       42]
CPU times: user 2.15 s, sys: 407 ms, total: 2.56 s
Wall time: 1.46 s


In [3]:
%%time

N = 50_000_000

# Create an array with numbers from 0 to N-1
# This array will also serve as our 'default' choice
source_array = np.arange(1, N + 1, dtype=np.int32)

# Define the conditions
# The order matters: np.select picks the first True condition
conditions = [
    (source_array % 15 == 0),  # Condition for FizzBuzz (divisible by 3 and 5)
    (source_array % 3 == 0),  # Condition for Fizz
    (source_array % 5 == 0),  # Condition for Buzz
]

# Define the corresponding choices for each condition
choices = [
    1337,  # Value for FizzBuzz
    17,  # Value for Fizz
    42,  # Value for Buzz
]

# Use np.select to create the output array
# If none of the conditions are met for an element,
# it will take the value from 'source_array' at that position.
output = np.select(conditions, choices, default=source_array)

print(output)

[       1        2       17 ...       17 49999999       42]
CPU times: user 494 ms, sys: 596 ms, total: 1.09 s
Wall time: 1.09 s


In [5]:
%%writefile sources/fizzbuzz_cupy_v1.py

import cupy as np

N = 50_000_000

source_array = np.arange(1, N + 1, dtype=np.int32)
output = np.copy(source_array)

divisible_by_15 = source_array % 15 == 0
output[divisible_by_15] = 1337

divisible_by_3_only = (source_array % 3 == 0) & (~divisible_by_15)
output[divisible_by_3_only] = 17

divisible_by_5_only = (source_array % 5 == 0) & (~divisible_by_15)
output[divisible_by_5_only] = 42

print(output)

Writing sources/fizzbuzz_cupy_v1.py


In [6]:
%%time
!python sources/fizzbuzz_cupy_v1.py

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/fizzbuzz_cupy_v1.py", line 2, in <module>
    import cupy as np
ModuleNotFoundError: No module named 'cupy'
CPU times: user 6.01 ms, sys: 10.3 ms, total: 16.3 ms
Wall time: 231 ms


In [7]:
%%writefile sources/fizzbuzz_cupy_v2.py

import cupy as np

N = 50_000_000

source_array = np.arange(1, N + 1, dtype=np.int32)

conditions = [
    (source_array % 15 == 0),  # Condition for FizzBuzz (divisible by 3 and 5)
    (source_array % 3 == 0),  # Condition for Fizz
    (source_array % 5 == 0),  # Condition for Buzz
    np.logical_or(source_array % 3 != 0, source_array % 5 != 0),
]

choices = [
    np.full(N, 1337),  # Value for FizzBuzz
    np.full(N, 17),  # Value for Fizz
    np.full(N, 42),  # Value for Buzz
    source_array,
]

output = np.select(conditions, choices)

print(output)

Writing sources/fizzbuzz_cupy_v2.py


In [8]:
%%time
!python sources/fizzbuzz_cupy_v2.py

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/fizzbuzz_cupy_v2.py", line 2, in <module>
    import cupy as np
ModuleNotFoundError: No module named 'cupy'
CPU times: user 1.33 ms, sys: 9.43 ms, total: 10.8 ms
Wall time: 140 ms


In [11]:
%%writefile sources/fizzbuzz_warp.py

import numpy as np
import warp as wp 

N = 50_000_000

@wp.kernel
def fizzbuzz(input: wp.array(dtype=wp.int32), output: wp.array(dtype=wp.int32)):
    i = wp.tid()
    if (i + 1) % 15 == 0:  # Condition for FizzBuzz (divisible by 3 and 5)
        output[i] = 1337
    elif (i + 1) % 3 == 0:  # Condition for Fizz
        output[i] = 17
    elif (i + 1) % 5 == 0:  # Condition for Buzz
        output[i] = 42
    else:
        output[i] = i + 1

# Allocate data for the input array
input_arra = wp.array(np.arnage(1, N+1), dtype=wp.int32)

# allocate daata for output array
output_arr = wp.empty_like(input_arr)

# launch the kernel
wp.launch(fizzbuzz, dim=(N, ), inputs=[input_arr, output_arr])
print(output_arr)

Overwriting sources/fizzbuzz_warp.py


In [12]:
%%time
!python sources/fizzbuzz_warp.py

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/fizzbuzz_warp.py", line 20, in <module>
    input_arra = wp.array(np.arnage(1, N+1), dtype=wp.int32)
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/dev_venv/lib/python3.10/site-packages/numpy/__init__.py", line 414, in __getattr__
    raise AttributeError("module {!r} has no attribute "
AttributeError: module 'numpy' has no attribute 'arnage'. Did you mean: 'arange'?
CPU times: user 4.63 ms, sys: 20.7 ms, total: 25.3 ms
Wall time: 782 ms


In [14]:
!nsys profile --force-overwrite true -o 01.01.01-fizzbuzz_cupy_v2.nsys-rep python sources/fizzbuzz_cupy_v2.py
!nsys stats --force-export=true --report cuda_gpu_kern_sum 01.01.01-fizzbuzz_cupy_v2.nsys-rep

Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

[       1        2       17 ...       17 49999999       42]
Generating '/tmp/nsys-report-f701.qdstrm'
[1/1] [========================100%] 01.01.01-fizzbuzz_cupy_v2.nsys-rep
Generated:
    /media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/01.01.01-fizzbuzz_cupy_v2.nsys-rep
Generating SQLite file 01.01.01-fizzbuzz_cupy_v2.sqlite from 01.01.01-fizzbuzz_cupy_v2.nsys-rep
Processing [01.01.01-fizzbuzz_cupy_v2.sqlite] with [/usr/local/cuda-12.6/nsight-systems-2024.5.1/host-linux-x64/reports/cuda_gpu_kern_sum.py]... 

 ** CUDA GPU Kernel Summary (cuda_gpu_kern_sum):

 Time (%)  Total Time (ns)  Instances   Avg (ns)     Med (ns)    Min (ns)   Max (ns)   StdDev (ns)                Name               
 --------  ---------------  ---------  -----------  -----------  ---------  ---------  -----------  -------

In [15]:
!nsys profile --force-overwrite true -o 01.01.01-fizzbuzz_warp.nsys-rep python sources/fizzbuzz_warp.py
!nsys stats --force-export=true --report cuda_gpu_kern_sum 01.01.01-fizzbuzz_warp.nsys-rep

Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/fizzbuzz_warp.py", line 20, in <module>
    input_arra = wp.array(np.arnage(1, N+1), dtype=wp.int32)
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/dev_venv/lib/python3.10/site-packages/numpy/__init__.py", line 414, in __getattr__
    raise AttributeError("module {!r} has no attribute "
AttributeError: module 'numpy' has no attribute 'arnage'. Did you mean: 'arange'?
Generating '/tmp/nsys-report-6ff8.qdstrm'
[1/1] [========================100%] 01.01.01-fizzbuzz_warp.nsys-rep
Generated:
    /media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/01.01.01-fizzbuzz_warp.nsys-rep
Generating SQLite file 01.01

In [18]:
# Semi-Implicit Euler Integration

In [17]:
%%writefile sources/semi_implicit_integration_cupy.py

import cupy as cp

import nvtx

num_particles = 10_000_000  # Number of particles
num_steps = 100

mass = 0.1  # Mass per particle [kg]
g = 9.81  # Gravitational acceleration [m/s^2]
b = 0.05  # Drag Coefficient [kg/s]

dt = 0.01 * (2 * mass / b)

gravity = cp.array([0.0, 0.0, -g], dtype=cp.float32)

# Initial positions: random values between -1.0 and 1.0 for x, y, and z
positions = cp.random.uniform(low=-1.0, high=1.0, size=(num_particles, 3), dtype=cp.float32)

# Initial velocities: random values between -0.5 and 0.5 m/s for vx, vy, and vz
velocities = cp.random.uniform(low=-0.5, high=0.5, size=(num_particles, 3), dtype=cp.float32)

for step in range(num_steps):
    with nvtx.annotate("Step", color="rapids"):
        acceleration = (-b * velocities) / mass + gravity
        velocities += acceleration * dt
        positions += velocities * dt

# Make sure work is complete before terminating
cp.cuda.Stream.null.synchronize()

Writing sources/semi_implicit_integration_cupy.py


In [19]:
!nsys profile --force-overwrite true -o 01.01.01-semi_implicit_integration_cupy.nsys-rep python sources/semi_implicit_integration_cupy.py
!nsys stats --force-export=true -r nvtx_gpu_proj_sum 01.01.01-semi_implicit_integration_cupy.nsys-rep

Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/semi_implicit_integration_cupy.py", line 4, in <module>
    import nvtx
ModuleNotFoundError: No module named 'nvtx'
Generating '/tmp/nsys-report-1576.qdstrm'
[1/1] [========================100%] 01.01.01-semi_implicit_integration_cupy.nsys-rep
Generated:
    /media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/01.01.01-semi_implicit_integration_cupy.nsys-rep
Generating SQLite file 01.01.01-semi_implicit_integration_cupy.sqlite from 01.01.01-semi_implicit_integration_cupy.nsys-rep
Processing [01.01.01-semi_implicit_integration_cupy.sqlite] with [/usr/local/cuda-12.6/nsight-systems-2024.5.1/host-linux-x64/reports/nvtx_gpu_proj_sum.p

In [20]:
%%writefile sources/semi_implicit_integration_warp.py

import numpy as np

import warp as wp

import nvtx

num_particles = 10_000_000  # Number of particles
num_steps = 100

mass = 0.1  # Mass per particle [kg]
g = 9.81  # Gravitational acceleration [m/s^2]
b = 0.05  # Drag Coefficient [kg/s]

dt = 0.01 * (2 * mass / b)

gravity = wp.vec3([0.0, 0.0, -g])

# Initial positions: random values between -1.0 and 1.0 for x, y, and z
rng = np.random.default_rng(12345)
positions_np = rng.uniform(low=-1.0, high=1.0, size=(num_particles, 3))
positions = wp.array(positions_np, dtype=wp.vec3)

# Initial velocities: random values between -0.5 and 0.5 m/s for vx, vy, and vz
velocities_np = rng.uniform(low=-0.5, high=0.5, size=(num_particles, 3))
velocities = wp.array(velocities_np, dtype=wp.vec3)


@wp.kernel
def integrate(positions: wp.array(dtype=wp.vec3), velocities: wp.array(dtype=wp.vec3)):
    i = wp.tid()

    acceleration = (-b * velocities[i]) / mass + gravity
    velocities[i] += acceleration * dt
    positions[i] += velocities[i] * dt


for step in range(num_steps):
    with nvtx.annotate("Step", color="rapids"):
        wp.launch(integrate, dim=(num_particles,), inputs=[positions, velocities])

wp.synchronize()

Writing sources/semi_implicit_integration_warp.py


In [21]:
!nsys profile --force-overwrite true -o 01.01.01-semi_implicit_integration_warp.nsys-rep python sources/semi_implicit_integration_warp.py
!nsys stats --force-export=true -r nvtx_gpu_proj_sum 01.01.01-semi_implicit_integration_warp.nsys-rep

Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.

Traceback (most recent call last):
  File "/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/sources/semi_implicit_integration_warp.py", line 6, in <module>
    import nvtx
ModuleNotFoundError: No module named 'nvtx'
Generating '/tmp/nsys-report-9db1.qdstrm'
[1/1] [========================100%] 01.01.01-semi_implicit_integration_warp.nsys-rep
Generated:
    /media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/physic-simulation/warp_samples/land_2025/01.01-Basics/01.01.01-semi_implicit_integration_warp.nsys-rep
Generating SQLite file 01.01.01-semi_implicit_integration_warp.sqlite from 01.01.01-semi_implicit_integration_warp.nsys-rep
Processing [01.01.01-semi_implicit_integration_warp.sqlite] with [/usr/local/cuda-12.6/nsight-systems-2024.5.1/host-linux-x64/reports/nvtx_gpu_proj_sum.p